In this notebook, we construct the US Tresury Yield curve from the data provided by the US Dept. of Treasury. 
Please refer to the link below:
https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value_month=202405

UST provide par yields (also called CMT rates) in their data set. This makes the bootstrapping a bit tricky since we will have to construct hypothetical bonds and then use QuantLib's classes to construct the yield curve.

References:
1. UST Dept of Treasury website (https://home.treasury.gov/)
2. QuantLib Documentation & Cookbook

In [10]:
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt
import requests
import xml.etree.ElementTree as ET
from datetime import datetime

In [41]:
class USTsyYieldCurve:
    def __init__(self, date: ql.Date) -> None:
        self.as_of_date = date
        self.api_str = f"{self.as_of_date.year()}{self.as_of_date.month():02d}"
        self.api_url = f"https://home.treasury.gov/resource-center/data-chart-center/interest-rates/pages/xml?data=daily_treasury_yield_curve&field_tdr_date_value_month={self.api_str}"
        self.raw_data = {}
        self.par_yields = self.get_raw_xml_data()
        if self.raw_data is not None:
            print("Fetched raw data from USDT website!")
        else:
            raise Exception("Error fetching data from USDT website!")

    def get_raw_xml_data(self):
        response = requests.get(self.api_url)

        if response.status_code == 200:

            # Parse the XML content
            root = ET.fromstring(response.content)

            # Define namespaces
            namespaces = {
                'atom': 'http://www.w3.org/2005/Atom',
                'm': 'http://schemas.microsoft.com/ado/2007/08/dataservices/metadata',
                'd': 'http://schemas.microsoft.com/ado/2007/08/dataservices'
            }

            # Find all entry tags
            entries = root.findall('.//atom:entry', namespaces)

            # Iterate through each entry tag
            for entry in entries:
                # Find the properties inside content inside entry tags
                content = entry.find('atom:content', namespaces)
                properties = content.find('m:properties', namespaces)
                if properties is not None:
                    # Iterate through each property
                    # Extract the NEW_DATE property and convert it to a datetime object
                    new_date_str = properties.find('d:NEW_DATE', namespaces).text
                    new_date = datetime.strptime(new_date_str, "%Y-%m-%dT%H:%M:%S")

                    if ql.Date(new_date.day, new_date.month, new_date.year) == self.as_of_date:
                        for prop in properties:
                            if (prop.tag.startswith('{http://schemas.microsoft.com/ado/2007/08/dataservices}BC_') 
                            and prop.tag != '{http://schemas.microsoft.com/ado/2007/08/dataservices}BC_30YEARDISPLAY'):
                                tag = prop.tag.split('}')[1]  # Remove namespace from the tag
                                period_str = tag.split("BC_")[1].replace("MONTH","M").replace("YEAR","Y")
                                ql_period = ql.Period(period_str)
                                raw_yield = ql.SimpleQuote(float(prop.text))
                                self.raw_data.update({ql_period:raw_yield})                

        else:
            print("Failed to fetch XML data. Status code:", response.status_code)
        
    


In [43]:
# Fetch data and build the yield curve
as_of_date = ql.Date(20, 5, 2024)
us_yield_curve = USTsyYieldCurve(date=as_of_date)




{}
